In [1]:
# mediapipe_angle_store.py의 출력으로 나온 raw데이터를 이용해서
# 각 라벨별로 분류 후, 라벨별로 30 frame 씩 sequential하게
# (sliding window) 끊어 저장

In [2]:
import numpy as np
import time
created_time = int(time.time())

In [3]:
# prompt 전체 출력되게
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

In [4]:
### label - idx mapping정보 가져오기
import pickle
with open('data/label_to_idx.pickle', 'rb') as f:
    label_to_idx = pickle.load(f)
print(label_to_idx)

### raw data load
raw = np.load('data/raw1~3000_1741280908.npy')
raw = raw.reshape(302378, 199)

print(raw.shape)
seq = [[0] for _ in range(419)] # 419개의 개별적인 리스트가 생성

i=0
for angle in raw:
    idx = int(angle[-1])
    seq[idx].append(angle)
    i=i+1
print(i)


{'0': 0, '1': 1, '10': 2, '100': 3, '1000': 4, '10000': 5, '11': 6, '112': 7, '119': 8, '12': 9, '13': 10, '14': 11, '15': 12, '16': 13, '17': 14, '18': 15, '19': 16, '2': 17, '20': 18, '21': 19, '22': 20, '23': 21, '24': 22, '25': 23, '26': 24, '27': 25, '28': 26, '29': 27, '3': 28, '30': 29, '31': 30, '32': 31, '33': 32, '34': 33, '35': 34, '36': 35, '37': 36, '38': 37, '39': 38, '4': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '5': 50, '50': 51, '51': 52, '52': 53, '53': 54, '54': 55, '55': 56, '56': 57, '57': 58, '58': 59, '59': 60, '6': 61, '60': 62, '61': 63, '62': 64, '63': 65, '64': 66, '65': 67, '66': 68, '67': 69, '68': 70, '69': 71, '7': 72, '70': 73, '71': 74, '72': 75, '73': 76, '74': 77, '75': 78, '76': 79, '77': 80, '78': 81, '79': 82, '8': 83, '80': 84, '81': 85, '82': 86, '83': 87, '84': 88, '85': 89, '86': 90, '87': 91, '88': 92, '89': 93, '9': 94, '90': 95, '91': 96, '92': 97, '93': 98, '94': 99, '95': 100, 

In [5]:

print(len(seq))
print(len(seq[0]))
print(len(seq[1]))
print(len(seq[-1]))

419
1004
839
650


In [6]:
# 0,로 시작하는데 0 지우기기
for per_label in seq:
    del per_label[0]
print(seq[0])

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [7]:

# frame에서 angle합이 0인 경우 == 손이 화면에 표시가 안된경우
# 삭제
# 앞에서부터 삭제하면 del per_label[i]로 원소를 삭제할 때 리스트가 한 칸 앞으로 당겨짐.
# 하지만 i는 그대로 증가하기 때문에 다음 원소를 건너뛰는 문제 발생!
# reversed로 뒤에서부터 삭제
# 리스트의 마지막 요소는 라벨이기 때문에 빼고 sum
for per_label in seq:
    for i in reversed(range(len(per_label))):
        if per_label[i][:-1].sum() < 1:
            del per_label[i]
print(len(seq[0]))

350


In [8]:
# 확인
temp = np.array(seq[0])
print(np.sum(temp, axis = 1))

[ 282.09739912  834.41829526  906.55433031  907.42151316  802.2304758   808.37719254  720.01293004  728.94279271  654.77513381  639.80667245  642.68224132  637.40649686  633.90036404  642.42493646  641.4041199   645.35833061  628.64240339  622.48482642  629.27489677  622.6677176   619.54678394  626.80835352  631.80362337  631.40724127  629.82479919  625.73214832  626.01409191  633.5754341   627.93304316  620.79370258  623.02666806  622.17480649  636.97909514  676.200092    656.40308699  701.96903424  841.78887596  842.40620282  349.06110356  799.67631173  308.20062232  823.90318422  798.60608683  775.98147952  665.13432939  622.54088733  579.15545243  567.78276391  566.88316326  576.87898289  580.89752427  570.53158592  558.45272781  558.41646507  563.51602997  552.99455538  566.94569192  574.7198471   572.02713985  565.97413534  662.42200466  577.37816789  684.26264805  597.76955983  585.77534467  593.18147447  565.11180759  562.5371967   579.55405086  617.34791868  632.40681743  579.

In [9]:
# npy sequence로 저장
# sliding window 처럼 자름
full_seq_data = []
seq_length = 60
save_seq_path = f'data/seq1~3000_{seq_length}fps_{created_time}'
for per_label in seq: 
    for start in range(len(per_label)- seq_length):
        full_seq_data.append(per_label[start:start+seq_length])
    temp = np.array(full_seq_data)

full_seq_data = np.array(full_seq_data)
print(full_seq_data.shape)
np.save(save_seq_path, full_seq_data)

print("##### 저장 끝 #####")

(159020, 60, 199)
##### 저장 끝 #####
